In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import yfinance as yf
import pandas_ta as ta
import plotly.graph_objects as go

In [2]:
def get_data(ticker: str):
    df = yf.download(tickers=ticker, period='max', interval='1d', auto_adjust=False)
    df.columns = ['Adj Close', 'Close', 'High', 'Low', 'Open', 'Volume']
    df = df.reset_index()
    return df

In [3]:
df = get_data('BTC-USD')

[*********************100%***********************]  1 of 1 completed


In [4]:
def calculate_sma(df, length: int):
    return ta.sma(df.Close, length)

In [5]:
df['SMA'] = calculate_sma(df, 20)
df = df.dropna()

# 1 - Slope of MAs

In [6]:

def calculate_slope(series, period: int = 5):
    series = np.asarray(series)
    slopes = [0.0] * (period - 1)

    x = np.arange(period)

    for i in range(period - 1, len(series)):
        y = series[i - period + 1 : i + 1]

        # Linear regression slope
        slope = np.polyfit(x, y, 1)[0]
        
        if y[0] != 0:
            percent_slope = (slope / y[0]) * 100
        else:
            percent_slope = 0.0

        slopes.append(percent_slope)

    return slopes

In [7]:
df['Slope'] = calculate_slope(df['SMA'])

In [8]:
df[40:55]

,Date,Adj Close,Close,High,Low,Open,Volume,SMA,Slope
59,2014-11-15,376.132996,376.132996,405.528015,371.007996,399.649994,15727500,356.616002,0.738455
60,2014-11-16,387.881989,387.881989,390.799011,374.601990,374.730011,11905600,358.360651,0.586591
61,2014-11-17,387.407990,387.407990,410.199005,377.502014,388.348999,41518800,359.850150,0.466889
62,2014-11-18,375.197998,375.197998,392.402008,371.117004,387.785004,32222500,361.830499,0.444559
63,2014-11-19,380.554993,380.554993,386.480988,373.895996,373.895996,18931800,363.592999,0.488588
64,2014-11-20,357.839996,357.839996,382.024994,356.781006,380.307007,25233200,364.568948,0.450927
65,2014-11-21,350.847992,350.847992,357.878998,344.112000,357.878998,29850100,365.823898,0.408113
66,2014-11-22,352.920013,352.920013,364.841003,350.877991,351.604004,15273000,367.175299,0.357087
67,2014-11-23,367.572998,367.572998,370.845001,353.174988,353.174988,15151600,369.176250,0.378799
68,2014-11-24,376.901001,376.901001,387.209015,366.669006,366.947998,30930100,371.496700,0.472005


In [9]:
dfpl = df[1000:1500]

fig = go.Figure(
    data = [
        go.Candlestick(
            x = dfpl.index,
            open = dfpl.Open,
            high = dfpl.High,
            low = dfpl.Low,
            close = dfpl.Close,
            name = 'OHLC'
        )
    ]
)

fig.add_scatter(
    x = dfpl.index,
    y = dfpl.SMA,
    mode = 'markers',
    marker = dict(size = 5, color = 'MediumPurple'),
    name = 'Pivot'
)

fig.update_layout(xaxis_rangeslider_visible=False)
fig.show()

# 2 - Three MAs alignment

In [10]:
df['SMA_10'] = calculate_sma(df, 10)
df['SMA_20'] = calculate_sma(df, 20)
df['SMA_30'] = calculate_sma(df, 30)

In [11]:
df

,Date,Adj Close,Close,High,Low,Open,Volume,SMA,Slope,SMA_10,SMA_20,SMA_30
19,2014-10-06,330.079010,330.079010,345.134003,302.559998,320.389008,79011800,389.910399,0.000000,NaN,NaN,NaN
20,2014-10-07,336.187012,336.187012,339.247009,320.481995,330.584015,49199900,383.853049,0.000000,NaN,NaN,NaN
21,2014-10-08,352.940002,352.940002,354.364014,327.187988,336.115997,54736300,380.278049,0.000000,NaN,NaN,NaN
22,2014-10-09,365.026001,365.026001,382.726013,347.687012,352.747986,83641104,378.789549,0.000000,NaN,NaN,NaN
23,2014-10-10,361.562012,361.562012,375.066986,352.963013,364.687012,43665700,376.422450,-0.821712,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
4145,2026-01-22,89462.453125,89462.453125,90258.960938,88438.445312,89378.523438,35549685694,92332.466406,0.063118,93176.567969,92332.466406,90872.974479
4146,2026-01-23,89503.875000,89503.875000,91100.250000,88486.359375,89462.046875,38997586037,92277.500781,-0.000576,92594.777344,92277.500781,90936.038281
4147,2026-01-24,89110.734375,89110.734375,89811.609375,89044.289062,89506.148438,14558687712,92162.362891,-0.043652,91812.917969,92162.362891,90998.571354
4148,2026-01-25,86572.218750,86572.218750,89193.148438,86003.710938,89104.765625,36124986722,91796.846094,-0.139629,90915.021094,91796.846094,90974.264323


In [12]:
def determine_trend(df):
    if df['SMA_10'] > df['SMA_20'] > df['SMA_30']:
        return 2 # Uptrend
    elif df['SMA_10'] < df['SMA_20'] < df['SMA_30']:
        return 1 # Downtrend
    else:
        return 0 # No trend

df['Trend'] = df.apply(determine_trend, axis=1)

In [13]:
df

,Date,Adj Close,Close,High,Low,Open,Volume,SMA,Slope,SMA_10,SMA_20,SMA_30,Trend
19,2014-10-06,330.079010,330.079010,345.134003,302.559998,320.389008,79011800,389.910399,0.000000,NaN,NaN,NaN,0
20,2014-10-07,336.187012,336.187012,339.247009,320.481995,330.584015,49199900,383.853049,0.000000,NaN,NaN,NaN,0
21,2014-10-08,352.940002,352.940002,354.364014,327.187988,336.115997,54736300,380.278049,0.000000,NaN,NaN,NaN,0
22,2014-10-09,365.026001,365.026001,382.726013,347.687012,352.747986,83641104,378.789549,0.000000,NaN,NaN,NaN,0
23,2014-10-10,361.562012,361.562012,375.066986,352.963013,364.687012,43665700,376.422450,-0.821712,NaN,NaN,NaN,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4145,2026-01-22,89462.453125,89462.453125,90258.960938,88438.445312,89378.523438,35549685694,92332.466406,0.063118,93176.567969,92332.466406,90872.974479,2
4146,2026-01-23,89503.875000,89503.875000,91100.250000,88486.359375,89462.046875,38997586037,92277.500781,-0.000576,92594.777344,92277.500781,90936.038281,2
4147,2026-01-24,89110.734375,89110.734375,89811.609375,89044.289062,89506.148438,14558687712,92162.362891,-0.043652,91812.917969,92162.362891,90998.571354,0
4148,2026-01-25,86572.218750,86572.218750,89193.148438,86003.710938,89104.765625,36124986722,91796.846094,-0.139629,90915.021094,91796.846094,90974.264323,0


In [14]:
dfpl = df[1000:1500]

fig = go.Figure(
    data = [
        go.Candlestick(
            x = dfpl.index,
            open = dfpl.Open,
            high = dfpl.High,
            low = dfpl.Low,
            close = dfpl.Close,
            name = 'OHLC'
        )
    ]
)

fig.add_trace(
    go.Scatter(
        x = dfpl.index,
        y = dfpl['SMA_10'],
        mode = 'lines',
        line = dict(color='blue'),
        name = 'SMA 10'
    )
)

fig.add_trace(
    go.Scatter(
        x = dfpl.index,
        y = dfpl['SMA_20'],
        mode = 'lines',
        line = dict(color='red'),
        name = 'SMA 20'
    )
)

fig.add_trace(
    go.Scatter(
        x = dfpl.index,
        y = dfpl['SMA_30'],
        mode = 'lines',
        line = dict(color='green'),
        name = 'SMA 30'
    )
)

fig.update_layout(xaxis_rangeslider_visible=False)
fig.show()

# 3 - Candles above or below the MA curve

In [15]:
def check_candles(df, backcandles, ma_columns):
    categories = [0] * backcandles

    for i in range(backcandles, len(df)):
        close_window = df['Close'].iloc[i - backcandles:i]
        ma_window = df[ma_columns].iloc[i - backcandles:i]

        if (close_window > ma_window).all():
            categories.append(2)  # Uptrend
        elif (close_window < ma_window).all():
            categories.append(1)  # Downtrend
        else:
            categories.append(0)  # No trend

    return categories

df['Category'] = check_candles(df, 5, 'SMA_20')

In [16]:
df

,Date,Adj Close,Close,High,Low,Open,Volume,SMA,Slope,SMA_10,SMA_20,SMA_30,Trend,Category
19,2014-10-06,330.079010,330.079010,345.134003,302.559998,320.389008,79011800,389.910399,0.000000,NaN,NaN,NaN,0,0
20,2014-10-07,336.187012,336.187012,339.247009,320.481995,330.584015,49199900,383.853049,0.000000,NaN,NaN,NaN,0,0
21,2014-10-08,352.940002,352.940002,354.364014,327.187988,336.115997,54736300,380.278049,0.000000,NaN,NaN,NaN,0,0
22,2014-10-09,365.026001,365.026001,382.726013,347.687012,352.747986,83641104,378.789549,0.000000,NaN,NaN,NaN,0,0
23,2014-10-10,361.562012,361.562012,375.066986,352.963013,364.687012,43665700,376.422450,-0.821712,NaN,NaN,NaN,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4145,2026-01-22,89462.453125,89462.453125,90258.960938,88438.445312,89378.523438,35549685694,92332.466406,0.063118,93176.567969,92332.466406,90872.974479,2,0
4146,2026-01-23,89503.875000,89503.875000,91100.250000,88486.359375,89462.046875,38997586037,92277.500781,-0.000576,92594.777344,92277.500781,90936.038281,2,0
4147,2026-01-24,89110.734375,89110.734375,89811.609375,89044.289062,89506.148438,14558687712,92162.362891,-0.043652,91812.917969,92162.362891,90998.571354,0,0
4148,2026-01-25,86572.218750,86572.218750,89193.148438,86003.710938,89104.765625,36124986722,91796.846094,-0.139629,90915.021094,91796.846094,90974.264323,0,1


In [17]:
dfpl = df[1000:1500]

fig = go.Figure(
    data = [
        go.Candlestick(
            x = dfpl.index,
            open = dfpl.Open,
            high = dfpl.High,
            low = dfpl.Low,
            close = dfpl.Close,
            name = 'OHLC'
        )
    ]
)

fig.add_trace(
    go.Scatter(
        x = dfpl.index,
        y = dfpl['SMA_20'],
        mode = 'lines',
        line = dict(color='red'),
        name = 'SMA 20'
    )
)

fig.update_layout(xaxis_rangeslider_visible=False)
fig.show()

# 4 - Apply trend detection using the VWAP curve

In [18]:
df = yf.download(tickers='BTC-USD', period='7d', interval='15m', auto_adjust=False)
df.columns = ['Adj Close', 'Close', 'High', 'Low', 'Open', 'Volume']

[*********************100%***********************]  1 of 1 completed


In [19]:
df.ta.vwap(append=True)
df

,Adj Close,Close,High,Low,Open,Volume,VWAP_D
Datetime,,,,,,,
2026-01-20 00:00:00+00:00,92616.101562,92616.101562,92796.171875,92544.859375,92583.640625,0,NaN
2026-01-20 00:15:00+00:00,92393.765625,92393.765625,92630.335938,92300.734375,92630.335938,0,NaN
2026-01-20 00:30:00+00:00,92664.578125,92664.578125,92674.031250,92397.679688,92397.679688,0,NaN
2026-01-20 00:45:00+00:00,92606.875000,92606.875000,92683.539062,92550.445312,92654.960938,0,NaN
2026-01-20 01:00:00+00:00,92506.218750,92506.218750,92711.906250,92490.617188,92637.406250,0,NaN
...,...,...,...,...,...,...,...
2026-01-26 13:30:00+00:00,87914.367188,87914.367188,87943.046875,87792.015625,87801.593750,1025949696,87608.363037
2026-01-26 13:45:00+00:00,87878.054688,87878.054688,87931.015625,87855.062500,87931.015625,960479232,87617.556903
2026-01-26 14:00:00+00:00,87579.710938,87579.710938,87902.750000,87579.710938,87872.265625,623980544,87619.017082


In [20]:
df['Category'] = check_candles(df, 5, 'VWAP_D')

In [21]:
df[df['Category'] != 0]

,Adj Close,Close,High,Low,Open,Volume,VWAP_D,Category
Datetime,,,,,,,,
2026-01-20 04:15:00+00:00,92157.226562,92157.226562,92253.703125,92157.226562,92253.703125,5648384,92368.247023,1
2026-01-20 04:30:00+00:00,92178.507812,92178.507812,92327.445312,92144.085938,92239.140625,517212160,92306.740248,1
2026-01-20 04:45:00+00:00,91954.492188,91954.492188,92190.000000,91954.492188,92177.867188,11200512,92304.355549,1
2026-01-20 05:00:00+00:00,91332.273438,91332.273438,91988.406250,91332.273438,91988.406250,1203994624,91940.036848,1
2026-01-20 05:15:00+00:00,91645.335938,91645.335938,91645.335938,91332.031250,91350.601562,1079410688,91819.326680,1
...,...,...,...,...,...,...,...,...
2026-01-26 13:00:00+00:00,87664.195312,87664.195312,87741.375000,87638.093750,87711.570312,848994304,87595.663228,2
2026-01-26 13:15:00+00:00,87818.421875,87818.421875,87834.695312,87720.031250,87737.328125,327184384,87598.010758,2
2026-01-26 13:30:00+00:00,87914.367188,87914.367188,87943.046875,87792.015625,87801.593750,1025949696,87608.363037,2


In [23]:
dfpl = df[:]

fig = go.Figure(
    data = [
        go.Candlestick(
            x = dfpl.index,
            open = dfpl.Open,
            high = dfpl.High,
            low = dfpl.Low,
            close = dfpl.Close,
            name = 'OHLC'
        )
    ]
)

fig.add_trace(
    go.Scatter(
        x = dfpl.index,
        y = dfpl['VWAP_D'],
        mode = 'lines',
        line = dict(color='red'),
        name = 'VWAP'
    )
)

fig.update_layout(xaxis_rangeslider_visible=False)
fig.show()

# 5 - Trend confirmation using the ADX

In [24]:
df.ta.adx(append=True)

,ADX_14,ADXR_14_2,DMP_14,DMN_14
Datetime,,,,
2026-01-20 00:00:00+00:00,NaN,NaN,NaN,NaN
2026-01-20 00:15:00+00:00,NaN,NaN,NaN,NaN
2026-01-20 00:30:00+00:00,NaN,NaN,NaN,NaN
2026-01-20 00:45:00+00:00,NaN,NaN,NaN,NaN
2026-01-20 01:00:00+00:00,NaN,NaN,NaN,NaN
...,...,...,...,...
2026-01-26 13:30:00+00:00,12.408297,12.874608,476.668443,387.876374
2026-01-26 13:45:00+00:00,12.255589,12.414170,442.620698,360.170918
2026-01-26 14:00:00+00:00,12.771191,12.589744,411.004933,609.795987


In [25]:
df

,Adj Close,Close,High,Low,Open,Volume,VWAP_D,Category,ADX_14,ADXR_14_2,DMP_14,DMN_14
Datetime,,,,,,,,,,,,
2026-01-20 00:00:00+00:00,92616.101562,92616.101562,92796.171875,92544.859375,92583.640625,0,NaN,0,NaN,NaN,NaN,NaN
2026-01-20 00:15:00+00:00,92393.765625,92393.765625,92630.335938,92300.734375,92630.335938,0,NaN,0,NaN,NaN,NaN,NaN
2026-01-20 00:30:00+00:00,92664.578125,92664.578125,92674.031250,92397.679688,92397.679688,0,NaN,0,NaN,NaN,NaN,NaN
2026-01-20 00:45:00+00:00,92606.875000,92606.875000,92683.539062,92550.445312,92654.960938,0,NaN,0,NaN,NaN,NaN,NaN
2026-01-20 01:00:00+00:00,92506.218750,92506.218750,92711.906250,92490.617188,92637.406250,0,NaN,0,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
2026-01-26 13:30:00+00:00,87914.367188,87914.367188,87943.046875,87792.015625,87801.593750,1025949696,87608.363037,2,12.408297,12.874608,476.668443,387.876374
2026-01-26 13:45:00+00:00,87878.054688,87878.054688,87931.015625,87855.062500,87931.015625,960479232,87617.556903,2,12.255589,12.414170,442.620698,360.170918
2026-01-26 14:00:00+00:00,87579.710938,87579.710938,87902.750000,87579.710938,87872.265625,623980544,87619.017082,2,12.771191,12.589744,411.004933,609.795987


In [28]:
def generate_trend_signal(df, threshold=40):
    trend_signal = []
    for i in range(len(df)):
        if df['ADX_14'].iloc[i] > threshold:
            if df['DMP_14'].iloc[i] > df['DMN_14'].iloc[i]:
                trend_signal.append(2) # Confirmed Uptrend
            else:
                trend_signal.append(1) # Confirmed Downtrend
        else:
            trend_signal.append(0) # No trend
    return trend_signal

In [29]:
df['Trend_Signal'] = generate_trend_signal(df)

In [ ]:
df[df['Trend_Signal'] != 0]

,Adj Close,Close,High,Low,Open,Volume,VWAP_D,Category,ADX_14,ADXR_14_2,DMP_14,DMN_14,Trend_Signal
Datetime,,,,,,,,,,,,,
2026-01-20 06:45:00+00:00,90770.218750,90770.218750,91038.468750,90714.109375,91038.468750,560410624,91478.680387,1,55.794259,NaN,236.217708,1597.718117,1
2026-01-20 07:00:00+00:00,90962.843750,90962.843750,91013.882812,90816.757812,90816.757812,1458747392,91386.612127,1,57.111760,NaN,219.345014,1483.595394,1
2026-01-20 07:15:00+00:00,90967.437500,90967.437500,90967.437500,90896.414062,90944.203125,369192960,91368.534578,1,58.335153,57.064706,203.677513,1377.624294,1
2026-01-20 07:30:00+00:00,91099.046875,91099.046875,91113.296875,90917.281250,90966.726562,16855040,91367.929418,1,58.346576,57.729168,334.988494,1279.222559,1
2026-01-20 07:45:00+00:00,91076.867188,91076.867188,91197.242188,91065.835938,91093.273438,263315456,91360.739231,1,57.756780,58.045966,395.006057,1187.849519,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2026-01-26 00:00:00+00:00,86911.304688,86911.304688,86932.851562,86577.585938,86577.585938,470593536,86807.247396,1,51.817023,54.315509,824.071112,1049.307436,1
2026-01-26 00:15:00+00:00,86763.726562,86763.726562,86999.718750,86717.773438,86937.890625,387379200,86816.198723,0,48.678403,51.778253,832.076077,974.356905,1
2026-01-26 00:30:00+00:00,86545.195312,86545.195312,86778.179688,86545.195312,86778.179688,693436416,86729.780288,0,46.377820,49.097422,772.642072,1077.338108,1


In [36]:
df['Confirmed_Signal'] = df.apply(lambda row: row['Category'] if row['Category'] == row['Trend_Signal'] else 0, axis=1)

In [44]:
df[df['Confirmed_Signal'] != 0]

,Adj Close,Close,High,Low,Open,Volume,VWAP_D,Category,ADX_14,ADXR_14_2,DMP_14,DMN_14,Trend_Signal,Confirmed_Signal
Datetime,,,,,,,,,,,,,,
2026-01-20 06:45:00+00:00,90770.218750,90770.218750,91038.468750,90714.109375,91038.468750,560410624,91478.680387,1,55.794259,NaN,236.217708,1597.718117,1,1.0
2026-01-20 07:00:00+00:00,90962.843750,90962.843750,91013.882812,90816.757812,90816.757812,1458747392,91386.612127,1,57.111760,NaN,219.345014,1483.595394,1,1.0
2026-01-20 07:15:00+00:00,90967.437500,90967.437500,90967.437500,90896.414062,90944.203125,369192960,91368.534578,1,58.335153,57.064706,203.677513,1377.624294,1,1.0
2026-01-20 07:30:00+00:00,91099.046875,91099.046875,91113.296875,90917.281250,90966.726562,16855040,91367.929418,1,58.346576,57.729168,334.988494,1279.222559,1,1.0
2026-01-20 07:45:00+00:00,91076.867188,91076.867188,91197.242188,91065.835938,91093.273438,263315456,91360.739231,1,57.756780,58.045966,395.006057,1187.849519,1,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2026-01-25 23:00:00+00:00,86355.351562,86355.351562,86442.281250,86003.710938,86442.281250,610830336,87564.728609,1,58.628997,59.185466,416.505334,1411.371957,1,1.0
2026-01-25 23:15:00+00:00,86418.523438,86418.523438,86547.000000,86179.992188,86179.992188,615419904,87538.707390,1,57.687885,58.320040,491.473703,1310.559674,1,1.0
2026-01-25 23:30:00+00:00,86277.960938,86277.960938,86488.812500,86220.085938,86408.132812,351567872,87523.693459,1,56.813995,57.721496,456.368438,1216.948269,1,1.0


In [39]:
dfpl = df[:]

fig = go.Figure(
    data = [
        go.Candlestick(
            x = dfpl.index,
            open = dfpl.Open,
            high = dfpl.High,
            low = dfpl.Low,
            close = dfpl.Close,
            name = 'OHLC'
        )
    ]
)

fig.add_trace(
    go.Scatter(
        x = dfpl.index,
        y = dfpl['VWAP_D'],
        mode = 'lines',
        line = dict(color='red'),
        name = 'VWAP'
    )
)

fig.update_layout(xaxis_rangeslider_visible=False)
fig.show()